In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
cd ..

/home/suriya/dcu/Group-Project


## Gomez et al with Polynomial Encoding of Categorical variables

In [13]:
import category_encoders as ce

In [97]:
from src.dataset.tc4tl import make_datasets
from src.features.gomez import features, postproc
from src.features.common import postproc_categorical


(trainset, devset, testset), pipe = make_datasets(features, postproc_categorical, verbose=True, testing=0,
                                     tunables={
                                         "cg=Y" : {
                                             "TX" : -52,
                                             "N"  : 2.6
                                         },
                                         "cg=N" : {
                                             "TX" : -54,
                                             "N" : 2.1
                                         },
                                        "CategoricalEncoder" : ce.polynomial.PolynomialEncoder
                                     })

100%|████████████████████████████████████| 15552/15552 [00:42<00:00, 369.53it/s]


NO Pipe input given!


100%|████████████████████████████████████████| 935/935 [00:02<00:00, 386.24it/s]


Pipe input given!


100%|██████████████████████████████████████| 8423/8423 [00:20<00:00, 413.12it/s]


Pipe input given!


In [98]:
trainset.head()

,intercept,Cat:TXDevice_0,Cat:TXDevice_1,Cat:TXDevice_2,Cat:TXDevice_3,Cat:TXDevice_4,Cat:TXDevice_5,Cat:TXDevice_6,Cat:TXDevice_7,Cat:TXDevice_8,Cat:TXDevice_9,Cat:TXDevice_10,Cat:TXDevice_11,Cat:TXDevice_12,Cat:TXDevice_13,Cat:TXPower_0,Cat:TXPower_1,Cat:RXDevice_0,Cat:RXDevice_1,Cat:RXDevice_2,Cat:RXDevice_3,Cat:RXDevice_4,Cat:RXDevice_5,Cat:RXDevice_6,Cat:RXDevice_7,Cat:RXDevice_8,Cat:TXCarry_0,Cat:RXCarry_0,Cat:RXPose_0,Cat:TXPose_0,Num:PredictedDistance,Num:MeanRssi,Num:PathLossAttenuation,CoarseGrain,fileid,Distance,DistanceFloat,Cat:TXDevice,Cat:TXPower,Cat:RXDevice,Cat:TXCarry,Cat:RXCarry,Cat:RXPose,Cat:TXPose
0,1,-0.418330,0.472270,-0.456256,0.393641,-0.307724,0.219002,-0.141859,0.083314,-0.044039,0.020706,-0.008499,0.002954,-0.000820,0.000158,-7.071068e-01,0.408248,-0.495434,0.522233,-0.453425,0.336581,-0.214834,0.116775,-0.052694,0.018699,-0.004535,-0.707107,-0.707107,-0.707107,-0.707107,2.453292,-62.133482,33.133482,0,aaadbuvp_tc4tl20.csv,1.8,1.8,iPhoneX,12,iPhone8,pocket,hand,standing,standing
1,1,-0.358569,0.269869,-0.065179,-0.168703,0.351684,-0.438004,0.425576,-0.345159,0.239071,-0.141982,0.071638,-0.029959,0.009845,-0.002210,-5.551115e-17,-0.816497,-0.385337,0.174078,0.151142,-0.411377,0.501280,-0.428174,0.275179,-0.130893,0.040816,-0.707107,0.707107,-0.707107,-0.707107,1.147594,-55.255556,22.255556,1,aaamkcii_tc4tl20.csv,3.0,3.0,iPhone11,8,iPhoneXS,pocket,pocket,standing,standing
2,1,-0.298807,0.098606,0.175483,-0.341733,0.300960,-0.084231,-0.185507,0.379950,-0.436039,0.371110,-0.248910,0.132074,-0.053325,0.014367,-7.071068e-01,0.408248,-0.275241,-0.087039,0.377854,-0.317882,-0.035806,0.389249,-0.503518,0.373979,-0.163266,0.707107,-0.707107,-0.707107,-0.707107,2.481524,-62.289089,33.289089,1,aabqtowt_tc4tl20.csv,1.2,1.2,iPhone6Plus,12,iPhone11,hand,hand,standing,standing
3,1,-0.239046,-0.041518,0.290801,-0.276847,0.013526,0.269541,-0.338278,0.143740,0.166479,-0.392270,0.432254,-0.323223,0.170641,-0.057470,-7.071068e-01,0.408248,-0.275241,-0.087039,0.377854,-0.317882,-0.035806,0.389249,-0.503518,0.373979,-0.163266,-0.707107,0.707107,0.707107,0.707107,4.610273,-67.938259,38.938259,1,aadkjwss_tc4tl20.csv,1.8,1.8,iPhone7,12,iPhone11,pocket,pocket,sitting,sitting
4,1,-0.179284,-0.150504,0.305842,-0.097919,-0.230870,0.301702,-0.032737,-0.284733,0.318923,-0.036178,-0.307192,0.450234,-0.351947,0.158042,-7.071068e-01,0.408248,-0.495434,0.522233,-0.453425,0.336581,-0.214834,0.116775,-0.052694,0.018699,-0.004535,0.707107,-0.707107,-0.707107,0.707107,1.395231,-55.760797,26.760797,0,aafzrgzt_tc4tl20.csv,1.8,1.8,iPhone6s,12,iPhone8,hand,hand,standing,sitting


In [99]:
predictors = []
for col in trainset.columns:
    if "Num:" in col or "CoarseGrain" in col or "_" in col :
        predictors.append(col)
predictors        

['Cat:TXDevice_0',
 'Cat:TXDevice_1',
 'Cat:TXDevice_2',
 'Cat:TXDevice_3',
 'Cat:TXDevice_4',
 'Cat:TXDevice_5',
 'Cat:TXDevice_6',
 'Cat:TXDevice_7',
 'Cat:TXDevice_8',
 'Cat:TXDevice_9',
 'Cat:TXDevice_10',
 'Cat:TXDevice_11',
 'Cat:TXDevice_12',
 'Cat:TXDevice_13',
 'Cat:TXPower_0',
 'Cat:TXPower_1',
 'Cat:RXDevice_0',
 'Cat:RXDevice_1',
 'Cat:RXDevice_2',
 'Cat:RXDevice_3',
 'Cat:RXDevice_4',
 'Cat:RXDevice_5',
 'Cat:RXDevice_6',
 'Cat:RXDevice_7',
 'Cat:RXDevice_8',
 'Cat:TXCarry_0',
 'Cat:RXCarry_0',
 'Cat:RXPose_0',
 'Cat:TXPose_0',
 'Num:PredictedDistance',
 'Num:MeanRssi',
 'Num:PathLossAttenuation',
 'CoarseGrain']

In [78]:
from src.models.evaluate import dual_evaluation
from src.ndcf import compute_ndcf
from sklearn.ensemble import GradientBoostingClassifier
import pandas as pd


gbm = GradientBoostingClassifier(random_state=15)
report, sysout = dual_evaluation(pd.concat([trainset, devset]), testset, predictors, target="Distance", model=gbm)

In [79]:
print(report["cg=0"])

              precision    recall  f1-score   support

         1.8       0.85      0.84      0.85      1423
         4.5       0.84      0.86      0.85      1401

    accuracy                           0.85      2824
   macro avg       0.85      0.85      0.85      2824
weighted avg       0.85      0.85      0.85      2824



In [80]:
print(report["cg=1"])

              precision    recall  f1-score   support

         1.2       0.59      0.55      0.57      1407
         1.8       0.52      0.37      0.43      1408
         3.0       0.39      0.53      0.45      1385
         4.5       0.48      0.49      0.49      1399

    accuracy                           0.48      5599
   macro avg       0.50      0.48      0.48      5599
weighted avg       0.50      0.48      0.48      5599



In [86]:
ndcf = compute_ndcf(sysout)
ndcf.nDCF.mean()

0.5140380647669411

## Ordinal Encoding

In [109]:
(trainset, devset, testset), pipe = make_datasets(features, postproc_categorical, verbose=True, testing=0,
                                     tunables={
                                         "cg=Y" : {
                                             "TX" : -52,
                                             "N"  : 2.6
                                         },
                                         "cg=N" : {
                                             "TX" : -54,
                                             "N" : 2.1
                                         },
                                        "CategoricalEncoder" : ce.ordinal.OrdinalEncoder
                                     })

100%|████████████████████████████████████| 15552/15552 [00:42<00:00, 367.26it/s]


NO Pipe input given!


100%|████████████████████████████████████████| 935/935 [00:02<00:00, 399.86it/s]


Pipe input given!


100%|██████████████████████████████████████| 8423/8423 [00:19<00:00, 425.84it/s]

Pipe input given!


In [110]:
trainset

,Cat:TXDevice,Cat:TXPower,Cat:RXDevice,Cat:TXCarry,Cat:RXCarry,Cat:RXPose,Cat:TXPose,Num:PredictedDistance,Num:MeanRssi,Num:PathLossAttenuation,CoarseGrain,fileid,Distance,DistanceFloat
0,1,1,1,1,1,1,1,2.453292,-62.133482,33.133482,0,aaadbuvp_tc4tl20.csv,1.8,1.8
1,2,2,2,1,2,1,1,1.147594,-55.255556,22.255556,1,aaamkcii_tc4tl20.csv,3.0,3.0
2,3,1,3,2,1,1,1,2.481524,-62.289089,33.289089,1,aabqtowt_tc4tl20.csv,1.2,1.2
3,4,1,3,1,2,2,2,4.610273,-67.938259,38.938259,1,aadkjwss_tc4tl20.csv,1.8,1.8
4,5,1,1,2,1,1,2,1.395231,-55.760797,26.760797,0,aafzrgzt_tc4tl20.csv,1.8,1.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15547,1,1,1,1,1,1,1,2.682236,-62.998435,33.998435,1,zzwcoemu_tc4tl20.csv,1.2,1.2
15548,12,3,6,2,1,2,1,12.480233,-77.020677,43.020677,1,zzwoqxuc_tc4tl20.csv,4.5,4.5
15549,14,2,7,1,2,1,2,9.435060,-74.469638,41.469638,1,zzymwbbb_tc4tl20.csv,1.2,1.2
15550,12,3,6,2,1,2,1,2.977816,-64.321346,30.321346,0,zzzflikz_tc4tl20.csv,1.8,1.8


In [111]:
predictors = []
for col in trainset.columns:
    if "Num:" in col or "CoarseGrain" in col or "Cat:" in col :
        predictors.append(col)
predictors        

['Cat:TXDevice',
 'Cat:TXPower',
 'Cat:RXDevice',
 'Cat:TXCarry',
 'Cat:RXCarry',
 'Cat:RXPose',
 'Cat:TXPose',
 'Num:PredictedDistance',
 'Num:MeanRssi',
 'Num:PathLossAttenuation',
 'CoarseGrain']

In [113]:
import numpy as np
from tqdm.notebook import tqdm

seed=0
gbm = GradientBoostingClassifier(random_state=seed)
report, sysout = dual_evaluation(pd.concat([trainset, devset]), testset, predictors, target="Distance", model=gbm)
ndcf = compute_ndcf(sysout).nDCF.mean()
print(ndcf)

0.5204989228892449
